# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
import os
import joblib

from azureml.train.automl import AutoMLConfig
from pprint import pprint # Used in printing automl model parameters
from azureml.core import Model # Used to get model information

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'bluewins-experiment'

experiment=Experiment(ws, experiment_name)

from azureml.data.dataset_factory import TabularDatasetFactory

data_loc = "https://raw.githubusercontent.com/ahmedkhammessi/nd00333-capstone/master/starter_file/high_diamond_ranked_10min.csv"
ds = TabularDatasetFactory.from_delimited_files(data_loc)
data = ds.to_pandas_dataframe().dropna()

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [4]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'}

computetarget = 'aml-compute'

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    compute_target=computetarget,
                             task='classification',
                             training_data=ds,
                             label_column_name='blueWins',   
                             n_cross_validations=5,
                             **automl_settings
)

In [5]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [6]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…


Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Beginning model selection.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

*************************************************************************************

{'runId': 'AutoML_aeee6398-e6a3-4917-9f70-c2f1dcef4012',
 'target': 'aml-compute',
 'status': 'Completed',
 'startTimeUtc': '2021-01-12T15:50:38.228365Z',
 'endTimeUtc': '2021-01-12T16:17:22.474752Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'aml-compute',
  'AMLSettingsJsonString': '{"path":null,"name":"bluewins-experiment","subscription_id":"0ff03191-6a7e-4a54-9fd4-ac9b388f83e4","resource_group":"capstone","workspace_name":"capstone-ws","region":"westeurope","compute_target":"aml-compute","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"y_max":null,"num_classes":null,"featurization":"auto","_

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [7]:
# Get best run and model
best_run, fitted_model = remote_run.get_output()
print(best_run)

# Get all metrics of the best run
best_run_metrics = best_run.get_metrics()
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

Run(Experiment: bluewins-experiment,
Id: AutoML_aeee6398-e6a3-4917-9f70-c2f1dcef4012_73,
Type: azureml.scriptrun,
Status: Completed)
AUC_macro 0.8085701606709849
f1_score_weighted 0.7336830943502114
norm_macro_recall 0.4673248840877074
recall_score_micro 0.7336771895659304
AUC_micro 0.8084091207367786
precision_score_macro 0.7336510946441155
balanced_accuracy 0.7336624420438536
weighted_accuracy 0.7336916934409019
AUC_weighted 0.8085701606709849
precision_score_micro 0.7336771895659304
recall_score_macro 0.7336624420438536
precision_score_weighted 0.7339182332066543
average_precision_score_macro 0.8057359928088397
accuracy 0.7336771895659304
f1_score_macro 0.7335421210748632
matthews_correlation 0.46731351401366317
log_loss 0.5443172246958384
average_precision_score_micro 0.8053771637938519
recall_score_weighted 0.7336771895659304
average_precision_score_weighted 0.8060021022615314
f1_score_micro 0.7336771895659304
confusion_matrix aml://artifactId/ExperimentRun/dcid.AutoML_aeee6398-e6

In [21]:
# Print detailed parameters of the fitted model
def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators': list(
                e[0] for e in step[1].estimators), 'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0] + ' - ')
        else:
            pprint(step[1].get_params())
            print()

print_model(fitted_model)

datatransformer
{'enable_dnn': None,
 'enable_feature_sweeping': None,
 'feature_sweeping_config': None,
 'feature_sweeping_timeout': None,
 'featurization_config': None,
 'force_text_dnn': None,
 'is_cross_validation': None,
 'is_onnx_compatible': None,
 'logger': None,
 'observer': None,
 'task': None,
 'working_dir': None}

prefittedsoftvotingclassifier
{'estimators': ['40', '55', '20', '19', '53', '38', '33', '68', '48', '28'],
 'weights': [0.18181818181818182,
             0.09090909090909091,
             0.09090909090909091,
             0.09090909090909091,
             0.09090909090909091,
             0.09090909090909091,
             0.09090909090909091,
             0.09090909090909091,
             0.09090909090909091,
             0.09090909090909091]}

40 - standardscalerwrapper
{'class_name': 'StandardScaler',
 'copy': True,
 'module_name': 'sklearn.preprocessing._data',
 'with_mean': True,
 'with_std': False}

40 - lightgbmclassifier
{'boosting_type': 'goss',
 'class_w

In [8]:
#TODO: Save the best model
myModel = best_run.register_model(model_path='outputs/model.pkl', model_name='capstoneModel_automl',
                        tags={'Training context':'Auto ML'},
                        properties={'Accuracy': best_run_metrics['accuracy']})

print(myModel)

Model(workspace=Workspace.create(name='capstone-ws', subscription_id='0ff03191-6a7e-4a54-9fd4-ac9b388f83e4', resource_group='capstone'), name=capstoneModel_automl, id=capstoneModel_automl:1, version=1, tags={'Training context': 'Auto ML'}, properties={'Accuracy': '0.7336771895659304'})


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [19]:
# Download scoring file 
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'scoreScript.py')

# Download environment file
best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'envFile.yml')

from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(entry_script='scoreScript.py',
                                    environment=best_run.get_environment())

# deploy
from azureml.core.webservice import AciWebservice

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
service = Model.deploy(ws, "akhamessi-endpoint", [myModel], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)

print(service.scoring_uri)

print(service.swagger_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.......................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
http://04451749-ab91-4dd2-8e71-e9ef69a20c96.westeurope.azurecontainer.io/score
http://04451749-ab91-4dd2-8e71-e9ef69a20c96.westeurope.azurecontainer.io/swagger.json


TODO: In the cell below, send a request to the web service you deployed to test it.

In [20]:
import json

#Import test data
test_df = data.sample(5) # data is the pandas dataframe of the original data
label_df = test_df.pop('blueWins')

test_sample = json.dumps({'data': test_df.to_dict(orient='records')})

print(test_sample)
#####
import requests # Used for http post request

# Set the content type
headers = {'Content-type': 'application/json'}


response = requests.post(service.scoring_uri, test_sample, headers=headers)
print("response")
print(response.text)

{"data": [{"gameId": 4521969261, "blueWardsPlaced": 11, "blueWardsDestroyed": 0, "blueFirstBlood": 1, "blueKills": 5, "blueDeaths": 6, "blueAssists": 4, "blueEliteMonsters": 1, "blueDragons": 1, "blueHeralds": 0, "blueTowersDestroyed": 0, "blueTotalGold": 15548, "blueAvgLevel": 7.0, "blueTotalExperience": 18653, "blueTotalMinionsKilled": 181, "blueTotalJungleMinionsKilled": 68, "blueGoldDiff": -1238, "blueExperienceDiff": -1708, "blueCSPerMin": 18.1, "blueGoldPerMin": 1554.8, "redWardsPlaced": 17, "redWardsDestroyed": 1, "redFirstBlood": 0, "redKills": 6, "redDeaths": 5, "redAssists": 5, "redEliteMonsters": 0, "redDragons": 0, "redHeralds": 0, "redTowersDestroyed": 0, "redTotalGold": 16786, "redAvgLevel": 7.4, "redTotalExperience": 20361, "redTotalMinionsKilled": 219, "redTotalJungleMinionsKilled": 60, "redGoldDiff": 1238, "redExperienceDiff": 1708, "redCSPerMin": 21.9, "redGoldPerMin": 1678.6}, {"gameId": 4511803387, "blueWardsPlaced": 17, "blueWardsDestroyed": 2, "blueFirstBlood": 0,

TODO: In the cell below, print the logs of the web service and delete the service

In [16]:
print(service.get_logs())
service.delete()

2021-01-15T20:31:32,297542100+00:00 - rsyslog/run 
2021-01-15T20:31:32,300481500+00:00 - gunicorn/run 
2021-01-15T20:31:32,297492800+00:00 - nginx/run 
2021-01-15T20:31:32,327213100+00:00 - iot-server/run 
/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd